# Notebook 04 — ZuCo Models: Reading Cost & EEG vs KEC (v4.3, Upgraded)

**Purpose.** Validate KEC metrics (transition entropy, local curvature, meso-coherence) contra ZuCo (ET+EEG) com:
- OLS + **SE robustos clusterizados** (Subject)
- **FDR** (Benjamini–Hochberg)
- **Bootstrap paralelizado** (joblib)
- **MixedLM** (random intercepts Subject + variance components SentenceID)

**Inputs esperados:**
- `data/processed/kec/metrics_en.csv`
- `data/processed/zuco_aligned.csv`

**Saídas:**
- `figures/metrics/F2_reading_vs_KEC.png`
- `figures/metrics/F3_EEG_vs_KEC.png`
- `data/processed/models_reading_coeffs.csv`
- `data/processed/models_reading_coeffs_fdr.csv`
- `data/processed/boot_ols_ffd_entropy.csv` (se bootstrap executado)
- `data/processed/mixedlm_ffd_summary.txt` (se MixedLM executado)
- `reports/qa_kec_models.json`


In [ ]:
import os, time, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

SEED = 42
np.random.seed(SEED)

FIG_DIR = Path('figures/metrics'); FIG_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR = Path('data/processed'); PROC_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR = Path('reports'); REPORTS_DIR.mkdir(parents=True, exist_ok=True)

def heartbeat(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")
heartbeat('Environment ready')

## 1) Load & merge (KEC + ZuCo)

In [ ]:
kec_path = Path('data/processed/kec/metrics_en.csv')
zuco_path = Path('data/processed/zuco_aligned.csv')
if not kec_path.exists(): warnings.warn(f'Missing {kec_path}')
if not zuco_path.exists(): warnings.warn(f'Missing {zuco_path}')
kec_df = pd.read_csv(kec_path) if kec_path.exists() else pd.DataFrame()
zuco_df = pd.read_csv(zuco_path) if zuco_path.exists() else pd.DataFrame()

def normcols(df):
    df.columns=[c.strip() for c in df.columns]; return df
kec_df = normcols(kec_df); zuco_df = normcols(zuco_df)

kec_word_col = 'word' if 'word' in kec_df.columns else ('Word' if 'Word' in kec_df.columns else None)
zuco_word_col = 'Word' if 'Word' in zuco_df.columns else ('word' if 'word' in zuco_df.columns else None)

if len(kec_df)>0 and len(zuco_df)>0 and kec_word_col and zuco_word_col:
    merged = zuco_df.merge(kec_df, left_on=zuco_word_col, right_on=kec_word_col, how='left', suffixes=('', '_kec'))
else:
    merged = pd.DataFrame()

heartbeat(f'Merged: {merged.shape if len(merged)>0 else "N/A"}')
merged.head(3) if len(merged)>0 else merged

## 2) Prepare columns & transforms

In [ ]:
if len(merged)==0:
    heartbeat('Merged empty — fill data and rerun.')

cols = merged.columns.tolist()
ET_COLS = [c for c in ['FFD','GD','TRT','GPT'] if c in cols]
EEG_COLS = [c for c in ['ThetaPower','AlphaPower'] if c in cols]
KEC_COLS = [c for c in ['entropy','curvature','coherence'] if c in cols]
if 'LogFreq' in merged: merged = merged.rename(columns={'LogFreq':'log_freq'})

needed = set(ET_COLS + KEC_COLS + ['Subject','SentenceID'])
df = merged.copy()
if len(df)>0 and needed.issubset(df.columns):
    df = df.dropna(subset=list(needed))
else:
    df = merged

if 'Subject' in df: df['Subject']=df['Subject'].astype(str)
if 'SentenceID' in df: df['SentenceID']=df['SentenceID'].astype(str)

for resp in ['TRT','GPT']:
    if resp in df: df[f'log_{resp}'] = np.log1p(df[resp])

heartbeat(f'ET: {ET_COLS}; EEG: {EEG_COLS}; KEC: {KEC_COLS}')

## 3) OLS (clustered SE por Subject) + export coeficientes

In [ ]:
reading_results = []
if len(df)>0 and 'Subject' in df:
    resp_list = [r for r in ['FFD','GD','log_TRT','log_GPT'] if r in df.columns]
    for resp in resp_list:
        preds = [c for c in ['entropy','curvature','coherence','length','log_freq','surprisal'] if c in df.columns]
        if len(preds)<3:
            heartbeat(f'Predictors missing for {resp}; skipping.')
            continue
        formula = f"{resp} ~ " + " + ".join(preds)
        heartbeat(f'Fitting OLS: {formula}')
        try:
            model = smf.ols(formula, data=df).fit()
            rob = model.get_robustcov_results(cov_type='cluster', groups=df['Subject'])
            coefs = rob.params.rename('coef').to_frame(); coefs['se']=rob.bse; coefs['t']=rob.tvalues; coefs['p']=rob.pvalues; coefs['response']=resp
            reading_results.append(coefs.reset_index().rename(columns={'index':'term'}))
        except Exception as e:
            warnings.warn(f'OLS failed for {resp}: {e}')

if reading_results:
    reading_tbl = pd.concat(reading_results, ignore_index=True)
    out_csv = PROC_DIR/'models_reading_coeffs.csv'
    reading_tbl.to_csv(out_csv, index=False)
    heartbeat(f'Saved → {out_csv}')
else:
    heartbeat('No reading models fitted.')

## 4) F2 — Partial effect (FFD ~ entropy) com covariáveis medianas

In [ ]:
def partial_plot(df, response='FFD', xvar='entropy', controls=None, n_points=100, savepath=None):
    if controls is None: controls=[]
    need=[response,xvar]+controls
    if not set(need).issubset(df.columns):
        heartbeat(f'Missing columns for partial plot: {need}')
        return
    formula = response + ' ~ ' + xvar
    if controls: formula += ' + ' + ' + '.join(controls)
    mod = smf.ols(formula, data=df.dropna(subset=need)).fit()
    xgrid = np.linspace(df[xvar].quantile(0.05), df[xvar].quantile(0.95), n_points)
    base = {c: float(df[c].median()) for c in controls}
    grid_df = pd.DataFrame({xvar: xgrid, **base}); yhat = mod.predict(grid_df)
    plt.figure(); samp = df.dropna(subset=need).sample(min(2000,len(df)), random_state=SEED)
    plt.scatter(samp[xvar], samp[response], alpha=0.2); plt.plot(xgrid, yhat)
    plt.xlabel(xvar); plt.ylabel(response)
    ttl = ', '.join([f"{c}@median" for c in controls]) if controls else 'no controls'
    plt.title(f'Partial: {response} ~ {xvar} ({ttl})')
    if savepath: plt.tight_layout(); plt.savefig(savepath, dpi=150); heartbeat(f'Saved → {savepath}')
    plt.show()

if len(df)>0 and {'FFD','entropy'}.issubset(df.columns):
    controls = [c for c in ['length','log_freq'] if c in df.columns]
    partial_plot(df, response='FFD', xvar='entropy', controls=controls, savepath=FIG_DIR/'F2_reading_vs_KEC.png')
else:
    heartbeat('Skipping F2 — required columns missing.')

## 5) F3 — EEG vs entropy (por sujeito)

In [ ]:
def per_subject_trends(df, eeg_col='ThetaPower', xvar='entropy', savepath=None):
    need={'Subject',eeg_col,xvar}
    if not need.issubset(df.columns):
        heartbeat(f'Missing for F3: {need}'); return
    subs = sorted(df['Subject'].unique()); nsub=len(subs)
    if nsub==0: heartbeat('No subjects for F3'); return
    import math
    ncols=4; nrows=int(math.ceil(nsub/ncols))
    plt.figure(figsize=(12, 3*nrows))
    for i,s in enumerate(subs,1):
        ax = plt.subplot(nrows, ncols, i)
        sdf = df[df['Subject']==s].dropna(subset=[eeg_col,xvar])
        if len(sdf)<5: ax.set_title(f'{s} (few data)'); continue
        ax.scatter(sdf[xvar], sdf[eeg_col], alpha=0.3)
        try:
            m = smf.ols(f"{eeg_col} ~ {xvar}", data=sdf).fit()
            xs = np.linspace(sdf[xvar].min(), sdf[xvar].max(), 50)
            ys = m.params['Intercept'] + m.params[xvar]*xs
            ax.plot(xs, ys)
        except Exception:
            pass
        ax.set_title(str(s)); ax.set_xlabel(xvar); ax.set_ylabel(eeg_col)
    plt.tight_layout()
    if savepath: plt.savefig(savepath, dpi=150); heartbeat(f'Saved → {savepath}')
    plt.show()

if len(df)>0 and 'entropy' in df.columns and any(c in df.columns for c in ['ThetaPower','AlphaPower']):
    eegc = 'ThetaPower' if 'ThetaPower' in df.columns else 'AlphaPower'
    per_subject_trends(df, eeg_col=eegc, xvar='entropy', savepath=FIG_DIR/'F3_EEG_vs_KEC.png')
else:
    heartbeat('Skipping F3 — EEG/entropy missing.')

## 6) QA JSON

In [ ]:
qa = {
  'n_rows_merged': int(len(merged)) if len(merged)>0 else 0,
  'et_cols': ET_COLS if 'ET_COLS' in globals() else [],
  'eeg_cols': EEG_COLS if 'EEG_COLS' in globals() else [],
  'kec_cols': KEC_COLS if 'KEC_COLS' in globals() else [],
  'reading_coeffs_csv': 'data/processed/models_reading_coeffs.csv' if (PROC_DIR/'models_reading_coeffs.csv').exists() else None,
  'fig_F2': 'figures/metrics/F2_reading_vs_KEC.png' if (FIG_DIR/'F2_reading_vs_KEC.png').exists() else None,
  'fig_F3': 'figures/metrics/F3_EEG_vs_KEC.png' if (FIG_DIR/'F3_EEG_vs_KEC.png').exists() else None
}
with open(REPORTS_DIR/'qa_kec_models.json', 'w') as f:
    import json; json.dump(qa, f, indent=2)
heartbeat('Saved → reports/qa_kec_models.json')

## 7) (Opcional) MixedLM (random intercepts Subject + vc SentenceID)
Executa um modelo confirmatório para FFD com KEC + controles, contabilizando repetição por sujeito e sentenças.

In [ ]:
from statsmodels.regression.mixed_linear_model import MixedLM
def fit_mixedlm_ffd(df):
    cols_req = {'FFD','entropy','curvature','coherence','length','log_freq','Subject','SentenceID'}
    if not cols_req.issubset(df.columns):
        print('MixedLM skipped — columns missing.'); return None
    mdf = df[list(cols_req)].dropna().copy()
    exog = sm.add_constant(mdf[['entropy','curvature','coherence','length','log_freq']])
    endog = mdf['FFD']
    vc = {'Sentence': '0 + C(SentenceID)'}
    model = MixedLM(endog, exog, groups=mdf['Subject'], vc_formula=vc)
    try:
        result = model.fit(reml=True, method='lbfgs')
        print(result.summary())
        out_txt = Path('data/processed')/'mixedlm_ffd_summary.txt'
        with open(out_txt, 'w') as f: f.write(str(result.summary()))
        print(f'Saved MixedLM summary → {out_txt}')
        return result
    except Exception as e:
        print(f'MixedLM failed: {e}'); return None

# Uso: result = fit_mixedlm_ffd(df)

## 8) (Opcional) Bootstrap paralelizado (joblib)
Bootstrap por cluster (Subject) para CIs de FFD ~ entropy + length + log_freq.

In [ ]:
from joblib import Parallel, delayed
def _boot_fit_once(groups, df, formula, rng):
    samp_groups = rng.choice(groups, size=len(groups), replace=True)
    samp_df = pd.concat([df[df['Subject']==g] for g in samp_groups], ignore_index=True)
    try:
        m = smf.ols(formula, data=samp_df).fit()
        return m.params
    except Exception:
        return None

def bootstrap_coefs_parallel(df, formula, group_col='Subject', B=1000, n_jobs=-1, seed=42):
    rng = np.random.default_rng(seed)
    groups = df[group_col].unique()
    seeds = rng.integers(low=0, high=2**32-1, size=B, endpoint=True)
    results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(_boot_fit_once)(groups, df, formula, np.random.default_rng(int(s))) for s in seeds
    )
    params = [r for r in results if r is not None]
    return pd.DataFrame(params)

# Exemplo (execute manualmente se quiser rodar):
# if len(df)>0 and {'FFD','entropy','length','log_freq','Subject'}.issubset(df.columns):
#     boot = bootstrap_coefs_parallel(df[['FFD','entropy','length','log_freq','Subject']].dropna(),
#                                     'FFD ~ entropy + length + log_freq',
#                                     B=1000, n_jobs=-1)
#     if len(boot)>0:
#         boot.to_csv('data/processed/boot_ols_ffd_entropy.csv', index=False)
#         heartbeat('Saved bootstrap → data/processed/boot_ols_ffd_entropy.csv')

## 9) FDR correction (Benjamini–Hochberg)
Ajusta p-values por família (FFD, GD, log_TRT, log_GPT).

In [ ]:
from statsmodels.stats.multitest import multipletests
reading_csv = PROC_DIR/'models_reading_coeffs.csv'
if reading_csv.exists():
    dfc = pd.read_csv(reading_csv)
    outs = []
    for resp, grp in dfc.groupby('response'):
        pvals = grp['p'].values
        rej, p_adj, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')
        g2 = grp.copy(); g2['p_fdr_bh']=p_adj; g2['rej_fdr_bh_0.05']=rej
        outs.append(g2)
    df_fdr = pd.concat(outs, ignore_index=True)
    out_csv = PROC_DIR/'models_reading_coeffs_fdr.csv'
    df_fdr.to_csv(out_csv, index=False)
    heartbeat(f'Saved FDR → {out_csv}')
else:
    heartbeat('FDR skipped — models_reading_coeffs.csv not found.')